In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./4.phd7_hhy5.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: './4.phd7_hhy5.csv'

In [ ]:
# 假定车辆为正方形，此时考虑切线只用考虑四个角
# 从车的x，y，yaw，length，width获取四个角的坐标
def get_four_corners(x, y, yaw, length, width):
    half_length = length / 2
    half_width = width / 2
    corners = np.array([[-half_length, half_width], [half_length, half_width], [half_length, -half_width], [-half_length, -half_width]])
    corners = np.dot(corners, np.array([[np.cos(yaw), np.sin(yaw)], [np.sin(yaw), -np.cos(yaw)]]).T)
    corners += np.array([x, y])
    return corners

print(get_four_corners(0, 0, np.pi/4, 4, 0))

In [ ]:
# 获取七个检查点的坐标
def get_seven_checkpoints(x, y, yaw, length, width):
    half_length = length / 2
    half_width = width / 2
    checkpoints = np.array([[-half_length, half_width], [half_length, half_width], [half_length, -half_width], [-half_length, -half_width], [half_length,0], [0,half_width], [0,-half_width]])
    checkpoints = np.dot(checkpoints, np.array([[np.cos(yaw), np.sin(yaw)], [np.sin(yaw), -np.cos(yaw)]]).T)
    checkpoints += np.array([x, y])
    return checkpoints

print(get_seven_checkpoints(0, 0, 0, 4, 2))

In [ ]:
def get_corners_and_checkpoints(df):
    four_corners_list = []
    seven_checkpoints_list = []
    for i in range(int(len(df)/2)):
        four_corners = get_four_corners(df['CoG position/X'][2*i], df['CoG position/Y'][2*i], df['CoG position/Yaw'][2*i], df['Length (m)'][2*i], df['Width (m)'][2*i])
        four_corners_list.append(four_corners)
        seven_checkpoints = get_seven_checkpoints(df['CoG position/X'][2*i+1], df['CoG position/Y'][2*i+1], df['CoG position/Yaw'][2*i+1], df['Length (m)'][2*i+1], df['Width (m)'][2*i+1])
        seven_checkpoints_list.append(seven_checkpoints)
    return four_corners_list, seven_checkpoints_list

get_corners_and_checkpoints(df)[0]

In [ ]:
def get_collision(df):   
    four_corners_list, seven_checkpoints_list = get_corners_and_checkpoints(df)
    if_collision_list = [False for i in range(int(len(df)/2)-1)]
    for frame in range(int(len(df)/2)-1):
        four_corners_after = four_corners_list[frame]
        four_corners_before = four_corners_list[frame+1]
        alpha_condition = False
        beta_condition = False
        for checkpoint_index in range(7):
            checkpoint_after = seven_checkpoints_list[frame][checkpoint_index]
            checkpoint_before = seven_checkpoints_list[frame+1][checkpoint_index]
            # 计算checkpoint_after与four_corners_after中四个点的角度
            angle_list_after = []
            angle_list_before = []
            for corner_index in range(4):
                angle_after = np.arctan2(checkpoint_after[1]-four_corners_after[corner_index][1], checkpoint_after[0]-four_corners_after[corner_index][0])
                angle_before = np.arctan2(checkpoint_before[1]-four_corners_before[corner_index][1], checkpoint_before[0]-four_corners_before[corner_index][0])
                angle_list_after.append(angle_after)
                angle_list_before.append(angle_before)
            alpha_after, beta_after = get_alpha_beta(angle_list_after)
            alpha_before, beta_before = get_alpha_beta(angle_list_before)
            # 分别判定顺时针角度下的alpha是否变大，beta是否变小
            # 类似之前的判定方法
            if (alpha_after-alpha_before > 0 and alpha_after-alpha_before < np.pi) or (alpha_after-alpha_before < -np.pi and alpha_after-alpha_before > -2*np.pi):
                alpha_condition = True
            if (beta_after-beta_before < 0 and beta_after-beta_before > -np.pi) or (beta_after-beta_before > np.pi and beta_after-beta_before < 2*np.pi):
                beta_condition = True
        if alpha_condition and beta_condition:
            if_collision_list[frame] = True
    return if_collision_list

collision = get_collision(df)
# 输出为true的帧
for i in range(len(collision)):
    if collision[i]:
        print(i)